<a href="https://colab.research.google.com/github/pietrzakkuba/computer-assisted-translation-labs/blob/main/lab_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Komputerowe wspomaganie tłumaczenia

# Zajęcia 8 - urównoleglanie

Na poprzednich zajęciach poznaliśmy techniki pozyskiwania tekstu z Internetu. Jeśli uda nam się w ten sposób pozyskać tekst w jednym języku oraz jego tłumaczenie na inny język, jesteśmy tylko o krok od uzyskania najbardziej przydatnego zasobu z punktu widzenia wspomagania tłumaczenia - pamięci tłumaczeń. Krokiem tym jest automatyczne urównoleglanie tekstu.

Automatyczne urównoleglanie tekstu składa się z dwóch kroków:
1. Podziału tekstu źródłowego oraz docelowego na zdania.
2. Dopasowaniu zdań źródłowych do docelowych.

Zdania, o których mowa w punkcie 1., powinniśmy rozumieć jako segmenty, tj. niekoniecznie kompletne zdania w sensie gramatycznym. Standardowym sposobem podziału tekstu na segmenty jest dzielenie po znaku nowej linii lub zaraz po kropce, o ile jest ona częścią sekwencji: ".[spacja][Wielka litera]"

### Ćwiczenie 1: Zaimplementuj podstawowy algorytm segmentacji tekstu. Użyj odpowiedniego wyrażenia regularnego, łapiącego wielkie litery w dowolnym języku, np. "Ż" (użyj klasy unikodowej). Zwróć listę segmentów.

In [ ]:
import regex

def sentence_split(text):
    sentences = []
    sentence_end = None
    sentence_start = 0
    for match in regex.finditer('\. \p{Lu}', text):
        sentence_end = match.start() + 1
        sentences.append((text[sentence_start:sentence_end]))
        sentence_start = match.end() - 1
    sentences.append(text[sentence_start:])
    return sentences

sentence_split('Żółwie (nazwa systematyczna nieustalona, zobacz Nomenklatura) – rząd owodniowców z gromady zauropsydów (Sauropsida)[1] lub według linneuszowskiej klasyfikacji rząd z gromady gadów (Reptilia). Rząd żółwi dzieli się na 2 podrzędy, 14[2] rodzin i obejmuje 356 gatunków oraz 122 podgatunki żyjące w czasach nowożytnych, tj. od 1500 roku n.e. W tym czasie wyginęło 7 gatunków oraz 3 podgatunki[3].')

['Żółwie (nazwa systematyczna nieustalona, zobacz Nomenklatura) – rząd owodniowców z gromady zauropsydów (Sauropsida)[1] lub według linneuszowskiej klasyfikacji rząd z gromady gadów (Reptilia).',
 'Rząd żółwi dzieli się na 2 podrzędy, 14[2] rodzin i obejmuje 356 gatunków oraz 122 podgatunki żyjące w czasach nowożytnych, tj. od 1500 roku n.e.',
 'W tym czasie wyginęło 7 gatunków oraz 3 podgatunki[3].']

### Ćwiczenie 2: Uruchom powyższy algorytm na treści wybranej przez siebie strony internetowej (do ściągnięcia treści strony wykorzystaj kod z laboratoriów nr 7). Zidentyfikuj co najmniej dwa wyjątki od ogólnej reguły podziału na segmenty i ulepsz algorytm.

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_by_url(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    for script in soup.find_all(['script', 'style']):
        script.extract()
    text = soup.text.strip()
    return regex.sub(r"\s+", " ", text)

def sentence_split_enhanced(text):
    sentences = []
    sentence_end = None
    sentence_start = 0
    for match in regex.finditer('(?<!(mgr|dr|mjr|np|hab))\. \p{Lu}', text):
        sentence_end = match.start() + 1
        sentences.append((text[sentence_start:sentence_end]))
        sentence_start = match.end() - 1
    sentences.append(text[sentence_start:])
    return sentences

scraped_text = scrape_by_url('https://polszczyzna.pl/pisownia-skrotow-czyli-skracac-slowa-by-stracily-fasonu/')
print('PRZED:')
print(sentence_split(scraped_text))
print('PO:')
print(sentence_split_enhanced(scraped_text))

PRZED:
['Pisownia skrótów, czyli jak skracać słowa, by nie straciły fasonu SekcjeSklep Błędy językowe Błędy gramatyczne Błędy leksykalne Błędy stylistyczne Błędy ortograficzne Błędy interpunkcyjne Poradnia Ciekawostki Sprawdź pisownię Słownik Dyktanda i quizy Informacje Kontakt O nas Polityka prywatności Sekcje Sklep Błędy językowe Błędy gramatyczne Błędy leksykalne Błędy stylistyczne Błędy ortograficzne Błędy interpunkcyjne Poradnia Ciekawostki Sprawdź pisownię Słownik Dyktanda i quizy Informacje Kontakt O nas Polityka prywatności Sklep Błędy językowe Błędy gramatyczne Błędy leksykalne Błędy stylistyczne Błędy ortograficzne Błędy interpunkcyjne Poradnia Ciekawostki Sprawdź pisownię Słownik Dyktanda i quizy Informacje Kontakt O nas Polityka prywatności Sklep Błędy językowe Błędy gramatyczne Błędy leksykalne Błędy stylistyczne Błędy ortograficzne Błędy interpunkcyjne Poradnia językowa Ciekawostki Słownik Dyktanda i quizy Informacje Kontakt O nas Polityka prywatności Sprawdź pisownię 3.1

Po podziale tekstu na segmenty po stronie źródłowej oraz docelowej, możemy przystąpić do kroku drugiego - dopasowania segmentów. Głównym wyzwaniem tego kroku jest fakt, iż po stronie źródłowej może być inna liczba segmentów, niż po stronie docelowej. Takie rozbieżności są bardzo częste, a wynikają między innymi z:
* tłumaczenia jednego zdania źródłowego przy użyciu więcej niż jednego zdania
* tłumaczenia więcej niż jednego zdania źródłowego przy użyciu jednego zdania
* pominięcia zdania podczas tłumaczenia
* rozbieżności pomiędzy wersjami tekstu źródłowego i docelowego (np. tekst źródłowy mógł być modyfikowany po przetłumaczeniu i tłumaczenie nie zostało zaktualizowane)
* przetłumaczenia tekstu źródłowego tylko częściowo

Problemy te rozwiązwyane są na różne sposoby. Najpopularniejszym programem do przeprowadzania urównoleglania jest [Hunalign](https://github.com/danielvarga/hunalign). Wejściem do programu są dwa pliki, zawierające po jednym segmencie w linii. Wyjściem - plik urównoleglony w wewnętrznym formacie hunaligna.

### Ćwiczenie 3: Odnajdź dowolną stronę, która jest dostępna w wielu językach. Pobierz z tej strony tekst oryginalny (tylko ze strony głównej) oraz przetłumaczony na dowolny inny język. Przy użyciu Pythona przygotuj pliki dla Hunaligna i uruchom go.

Wyjściem z Hunaligna jest plik w specjalnym formacie Hunaligna. Problem jednak w tym, że niestety nie można go w prosty sposób zaimportować do jakiegokolwiek narzędzia typu CAT. Potrzebna jest konwersja do któregoś z bardziej popularnych formatów, np. XLIFF.

XLIFF jest formatem do przechowywania pamięci tłumaczeń, który opiera się na XML-u. Przykładowy plik XLIFF wygląda następująco:

<?xml version="1.0" encoding="UTF-8"?>
<xliff xmlns="urn:oasis:names:tc:xliff:document:1.2" version="1.2">
    <file datatype="plaintext" original="self" source-language="en" target-language="es">
        <header>
            <sxmd:metadata xmlns:sxmd="urn:x-sap:mlt:xliff12:metadata:1.0" xmlns="urn:x-sap:mlt:tsmetadata:1.0">
                <object-name>sample</object-name>
                <collection>KWT</collection>
                <domain>KWT</domain>
                <developer>123</developer>
                <description>sample XLIFF file</description>
            </sxmd:metadata>
        </header>
        <body>
            <trans-unit>
                <source>Hello world!</source>
                <target>Hola mundo!</target>
            </trans-unit>
            <trans-unit>
                <source>File</source>
                <target>Archivo</target>
            </trans-unit>
            <trans-unit>
                <source>New</source>
                <target>Nuevo</target>
            </trans-unit>
            <trans-unit>
                <source>Exit</source>
                <target>Salir</target>
            </trans-unit>
        </body>
    </file>
</xliff>

### Ćwiczenie 4: Napisz konwerter formatu hunaligna na XLIFF.

In [ ]:
def convert2xliff(hunalign_file_name):
    return 0